## Imports

In [5]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import tabula

## Read-in

In [6]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.io.read_pdf('condo.pdf', pages='all',lattice=True)

## Clean Data

In [7]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## Geocode

In [8]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [10]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [13]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [16]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction section

In [20]:
df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,geocoded,lat,lon
1,Continuum South Tower\r100 S Pointe Dr 1206\rM...,2/3/2023,95,"$10,250,000.00",2508,"$4,086.92",Bryan Sereny PA,Douglas Elliman,Matthew Wilkens,One Sotheby's Int'l Realty,Continuum South Tower 100 S Pointe Dr 1206 Mia...,Continuum South Tower,100 S Pointe Dr 1206 Miami Beach,"(25.7672315, -80.1327611)",25.767232,-80.132761
2,Murano at Portofino\r1000 S Pointe Dr 801\rMia...,2/3/2023,128,"$8,250,000.00",3365,"$2,451.71",Pablo Alfaro PA,Douglas Elliman,Ekaterina Sizenova,"Big International Realty, Inc.",Murano at Portofino 1000 S Pointe Dr 801 Miami...,Murano at Portofino,1000 S Pointe Dr 801 Miami Beach,"(25.7685326, -80.13755619999999)",25.768533,-80.137556
3,Jade Ocean\r17121 Collins Ave 4803\rSunny Isle...,2/1/2023,48,"$8,100,000.00",8332,$972.16,Anna Sherrill,One Sotheby's Int'l Realty,Nicolas Pollini PA,BHHS EWM Realty,Jade Ocean 17121 Collins Ave 4803 Sunny Isles ...,Jade Ocean,17121 Collins Ave 4803 Sunny Isles Beach,"(25.9344014, -80.1210054)",25.934401,-80.121005
4,Jade Signature\r16901 Collins Ave 4601\rSunny ...,1/30/2023,387,"$6,450,000.00",4361,"$1,479.02",Diane Lieberman,One Sotheby's Int'l Realty,Wendy Cohen,One Sotheby's Int'l Realty,Jade Signature 16901 Collins Ave 4601 Sunny Is...,Jade Signature,16901 Collins Ave 4601 Sunny Isles Beach,"(25.9327557, -80.1210054)",25.932756,-80.121005
5,Rise Condo\r88 SW 7th St PH 4204\rMiami,2/2/2023,17,"$6,400,000.00",3606,"$1,774.82",Allison Ortiz,Fortune Christie's Int'l Real Estate,Banna Fakhoury,One Sotheby's Int'l Realty,Rise Condo 88 SW 7th St PH 4204 Miami,Rise Condo,88 SW 7th St PH 4204 Miami,"(25.7670461, -80.1944977)",25.767046,-80.194498
6,Mansions at Acqualina\r17749 Collins Ave 2102\...,2/1/2023,292,"$6,250,000.00",4609,"$1,356.04",Michael Goldstein,Acqualina Realty,Michael Goldstein,Acqualina Realty,Mansions at Acqualina 17749 Collins Ave 2102 S...,Mansions at Acqualina,17749 Collins Ave 2102 Sunny Isles Beach,"(25.9400518, -80.12056079999999)",25.940052,-80.120561
7,Club Tower Two Condo\r785 Crandon Blvd 801\rKe...,1/30/2023,146,"$3,400,000.00",2790,"$1,218.64",Paulette Monserrate-Schena,BHHS EWM Realty,Michael P. Schnabel,Compass Florida LLC,Club Tower Two Condo 785 Crandon Blvd 801 Key ...,Club Tower Two Condo,785 Crandon Blvd 801 Key Biscayne,"(25.6875547, -80.15912399999999)",25.687555,-80.159124
8,Marquis Condo\r1100 Biscayne Blvd 5106\rMiami,1/31/2023,64,"$3,100,000.00",3800,$815.79,Oscar Teran,EXP Realty LLC,Nohemy Sanchez,"WorldWide Select Group, Inc",Marquis Condo 1100 Biscayne Blvd 5106 Miami,Marquis Condo,1100 Biscayne Blvd 5106 Miami,"(25.7851774, -80.1900783)",25.785177,-80.190078
9,Ritz Carlton Residences\r15701 Collins Ave 290...,1/31/2023,262,"$2,800,000.00",1447,"$1,935.04",Keila Nomura,IHB Brazil Realty LLC,Wendy Cohen,One Sotheby's Int'l Realty,Ritz Carlton Residences 15701 Collins Ave 2903...,Ritz Carlton Residences,15701 Collins Ave 2903 Sunny Isles Beach,"(25.922045, -80.12199729999999)",25.922045,-80.121997
10,0Lake Tower Condo\r765 Crandon Blvd 505\rKey B...,1/30/2023,199,"$2,750,000.00",2602,"$1,056.88",Paula Barreto Brook,BHHS EWM Realty,Ingo Viehweg,BHHS EWM Realty,0Lake Tower Condo 765 Crandon Blvd 505 Key Bis...,Lake Tower Condo,765 Crandon Blvd 505 Key Biscayne,"(25.688799, -80.15683539999999)",25.688799,-80.156835


In [19]:
df.at[10,'building_name']=('Lake Tower Condo')
df.at[10,'address_only']=('765 Crandon Blvd 505 Key Biscayne')

In [21]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only',
       'geocoded', 'lat', 'lon'],
      dtype='object')

In [27]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('January 29th - February 4th')

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r['building_name'],r['Sale Price'],r['days_on_market'],r['Listing Broker'],r['Buyer Agent'],r['price_per_sqft'],r['address_only'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[0]}{bold_end}{new_line}\
    Sale Price: {bold_start}{info[1]}{bold_end}{new_line}\
    Days on Market: {bold_start}{info[2]}{bold_end}{new_line}\
    Listing Agent: {bold_start}{info[3]}{bold_end}{new_line}\
    Buying Agent: {bold_start}{info[4]}{bold_end}{new_line}\
    Price sq ft: {bold_start}{info[5]}{bold_end}{new_line}\
    Address: {bold_start}{info[6]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [28]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [29]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [30]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_02042023
